## YOLO11 OBB Model Training and Data Preparation

##### This section of the notebook initializes the necessary libraries and sets up the environment for training a YOLO model using oriented bounding box (OBB) annotations.

In [2]:
from ultralytics import YOLO
import os
import xml.etree.ElementTree as ET
import shutil
import random
import matplotlib.pyplot as plt
from PIL import Image

## Convert XML to YOLO Oriented Bounding Box (OBB) Format

##### This Python script converts XML annotation files (Pascal VOC format) with oriented bounding boxes (OBB) into the YOLO OBB format. It reads annotation files from a specified directory, extracts object labels and bounding box coordinates, normalizes them relative to image dimensions, and saves them in YOLO OBB format. The script includes error handling for missing or invalid data and provides warnings for unrecognized classes. This tool is useful for preparing datasets for YOLO-based object detection models that support oriented bounding boxes.

In [ ]:
# Set paths
xml_dir = "dataset/Annotations/Oriented Bounding Boxes"  # Update this to your XML folder
output_dir = "dataset/Annotations/yolo-obb-format"  # Update this to your YOLO labels folder
os.makedirs(output_dir, exist_ok=True)

# Define class mapping (update based on your dataset)
class_mapping = {
    "A1": 0, "A10": 1, "A11": 2, "A12": 3, "A13": 4, "A14": 5, "A15": 6, "A16": 7, "A17": 8, "A18": 9,
    "A19": 10, "A2": 11, "A20": 12, "A3": 13, "A4": 14, "A5": 15, "A6": 16, "A7": 17, "A8": 18, "A9": 19
}

def convert_xml_to_yolo_obb(xml_file, output_folder):
    """ Convert XML annotation to YOLO OBB format. """
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Get image filename (without extension)
    image_filename = root.find("filename").text
    image_name = os.path.splitext(image_filename)[0]

    # Get image dimensions (handling missing values)
    size = root.find("size")
    if size is None:
        print(f"⚠️ Warning: Missing <size> tag in {xml_file}. Skipping file.")
        return
    
    width = size.find("width")
    height = size.find("height")

    # Ensure width and height exist and are valid numbers
    try:
        image_width = int(width.text) if width is not None else None
        image_height = int(height.text) if height is not None else None

        if not image_width or not image_height:
            print(f"⚠️ Warning: Missing or invalid image dimensions in {xml_file}. Skipping file.")
            return

    except ValueError:
        print(f"⚠️ Warning: Invalid width/height in {xml_file}. Skipping file.")
        return

    # Prepare output file
    output_path = os.path.join(output_folder, f"{image_name}.txt")
    yolo_annotations = []

    # Iterate over each object in the XML file
    for obj in root.findall("object"):
        class_name = obj.find("name").text

        # Check if class exists in mapping
        if class_name not in class_mapping:
            print(f"⚠️ Warning: Class '{class_name}' not found in class mapping. Skipping...")
            continue

        class_id = class_mapping[class_name]

        # Get OBB coordinates
        robndbox = obj.find("robndbox")
        try:
            x1 = float(robndbox.find("x_left_top").text) / image_width
            y1 = float(robndbox.find("y_left_top").text) / image_height
            x2 = float(robndbox.find("x_right_top").text) / image_width
            y2 = float(robndbox.find("y_right_top").text) / image_height
            x3 = float(robndbox.find("x_right_bottom").text) / image_width
            y3 = float(robndbox.find("y_right_bottom").text) / image_height
            x4 = float(robndbox.find("x_left_bottom").text) / image_width
            y4 = float(robndbox.find("y_left_bottom").text) / image_height

            # Format for YOLO OBB
            yolo_annotations.append(f"{class_id} {x1:.6f} {y1:.6f} {x2:.6f} {y2:.6f} {x3:.6f} {y3:.6f} {x4:.6f} {y4:.6f}")

        except (AttributeError, ValueError):
            print(f"⚠️ Warning: Missing or invalid bounding box values in {xml_file}. Skipping object.")

    # Save annotations to YOLO OBB text file
    if yolo_annotations:
        with open(output_path, "w") as f:
            f.write("\n".join(yolo_annotations))
        print(f"✅ Converted: {xml_file} → {output_path}")
    else:
        print(f"⚠️ No valid annotations found in {xml_file}. Skipping file.")

# Process all XML files
xml_files = [f for f in os.listdir(xml_dir) if f.endswith(".xml")]
for xml_file in xml_files:
    convert_xml_to_yolo_obb(os.path.join(xml_dir, xml_file), output_dir)

print("🚀 XML to YOLO OBB conversion complete!")


## YOLO-OBB Dataset Preparation: Train-Validation Split Script

##### This script organizes an object detection dataset by splitting images and their corresponding YOLO-OBB format annotations into training and validation sets. It randomly shuffles the dataset and separates it based on a predefined ratio (80% training, 20% validation). The processed files are then moved to structured directories under Processed_dataset, ensuring easy accessibility for model training.

In [ ]:
# Set dataset paths
images_dir = r"dataset/Images"  # Folder containing images
labels_dir = r"dataset/Annotations/yolo-obb-format"  # Folder containing text label files
output_dir = r"dataset/Processed_dataset"  # Output directory

# Train-validation split ratio
train_ratio = 0.8  # 80% train
val_ratio = 0.2    # 20% validation

# Ensure ratios sum to 1
assert train_ratio + val_ratio == 1.0, "Train and validation ratios must sum to 1.0"

# Create output directories
for split in ["train", "val"]:
    os.makedirs(os.path.join(output_dir, "images", split), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "labels", split), exist_ok=True)

# Collect all image files (supporting .jpg, .png, .jpeg)
all_images = [f for f in os.listdir(images_dir) if f.endswith((".jpg", ".png", ".jpeg"))]
random.shuffle(all_images)  # Shuffle dataset for randomness

# Calculate split indices
total_images = len(all_images)
train_end = int(total_images * train_ratio)

train_images = all_images[:train_end]
val_images = all_images[train_end:]

def move_files(image_list, split):
    """ Moves images and corresponding labels to destination folders """
    img_dest = os.path.join(output_dir, "images", split)
    lbl_dest = os.path.join(output_dir, "labels", split)

    for img_file in image_list:
        img_path = os.path.join(images_dir, img_file)
        lbl_path = os.path.join(labels_dir, os.path.splitext(img_file)[0] + ".txt")  # Corresponding label file

        # Move image
        shutil.move(img_path, os.path.join(img_dest, img_file))

        # Move label if exists
        if os.path.exists(lbl_path):
            shutil.move(lbl_path, os.path.join(lbl_dest, os.path.basename(lbl_path)))

# Move images & labels to respective folders
move_files(train_images, "train")
move_files(val_images, "val")

print(f"✅ Dataset split completed! 🚀")
print(f"🟢 Training set: {len(train_images)} images")
print(f"🔵 Validation set: {len(val_images)} images")


## Creating dataset.yaml File

The `dataset.yaml` file defines the structure of our dataset, specifying the paths to training and validation images along with the class names for object detection. This file is essential for configuring object detection models like YOLO.

#### `dataset.yaml` File Structure:
```yaml
train: dataset/Processed_dataset/images/train
val: dataset/Processed_dataset/images/val

names:
  0: A1
  1: A10
  2: A11
  3: A12
  4: A13
  5: A14
  6: A15
  7: A16
  8: A17
  9: A18
  10: A19
  11: A2
  12: A20
  13: A3
  14: A4
  15: A5
  16: A6
  17: A7
  18: A8
  19: A9
```

## Training a YOLO11n-OBB Model on the Dataset

##### This code initializes a YOLO11n-OBB model from scratch using a specified configuration file (yolo11n-obb.yaml). The model is then trained on the [Military Aircraft Recognition dataset](https://www.kaggle.com/datasets/khlaifiabilel/military-aircraft-recognition-dataset) for 50 epochs with an image size of 1024x1024 and a batch size of 4. The validation step is disabled (val=False) during training.

In [ ]:
# Create a new YOLO11n-OBB model from scratch
model = YOLO("yolo11n-obb.yaml")

# Train the model on the DOTAv1 dataset
results = model.train(data="dataset.yaml", epochs=50, imgsz=1024, batch=4, val=False)

## Validating the YOLO11n-OBB Model on the Dataset

In [ ]:
# Validate the model
metrics = model.val(data="dataset.yaml")  # no arguments needed, dataset and settings remembered

# YOLO11n-OBB Training Summary

**Environment:**
- **Ultralytics Version:** 8.3.70  
- **Python Version:** 3.9.21  
- **Torch Version:** 2.5.1+cu121  
- **GPU:** NVIDIA GeForce RTX 3050 Laptop GPU (4096MiB)  

## Model Summary
- **YOLO11n-OBB (Fused):** 257 layers, 2,657,623 parameters, 0 gradients, 6.6 GFLOPs  

---

## Validation Results
- **Total Images Scanned:** 768  
- **Background Images:** 1  
- **Corrupt Images:** 0  

### Performance Metrics:

| Class  | Images | Instances | Precision (P) | Recall (R) | mAP@50 | mAP@50-95 |
|--------|--------|-----------|--------------|------------|--------|------------|
| **All** | 769 | 4354 | **0.896** | **0.858** | **0.928** | **0.771** |
| A1  | 64 | 277 | 0.82 | 0.856 | 0.926 | 0.756 |
| A10 | 42 | 213 | 0.962 | 0.949 | 0.985 | 0.851 |
| A11 | 48 | 127 | 0.793 | 0.772 | 0.87  | 0.785 |
| A12 | 35 | 112 | 0.961 | 0.786 | 0.908 | 0.684 |
| A13 | 62 | 385 | 0.867 | 0.914 | 0.947 | 0.716 |
| A14 | 96 | 358 | 0.933 | 0.944 | 0.979 | 0.866 |
| A15 | 27 | 108 | 0.717 | 0.656 | 0.73  | 0.536 |
| A16 | 58 | 518 | 0.946 | 0.942 | 0.98  | 0.779 |
| A17 | 66 | 253 | 0.927 | 0.988 | 0.978 | 0.864 |
| A18 | 20 | 57  | 0.844 | 0.649 | 0.795 | 0.7   |
| A19 | 66 | 211 | 0.842 | 0.659 | 0.852 | 0.661 |
| A2  | 78 | 301 | 0.961 | 0.973 | 0.99  | 0.832 |
| A20 | 44 | 198 | 0.921 | 0.821 | 0.907 | 0.597 |
| A3  | 70 | 285 | 0.961 | 0.942 | 0.985 | 0.845 |
| A4  | 31 | 108 | 0.85  | 0.895 | 0.937 | 0.825 |
| A5  | 48 | 234 | 0.943 | 0.712 | 0.911 | 0.696 |
| A6  | 18 | 71  | 0.876 | 0.93  | 0.967 | 0.866 |
| A7  | 45 | 149 | 0.913 | 0.799 | 0.932 | 0.844 |
| A8  | 37 | 181 | 0.949 | 0.994 | 0.994 | 0.853 |
| A9  | 45 | 208 | 0.928 | 0.981 | 0.988 | 0.864 |

---

## Processing Speed
- **Preprocessing:** 1.4ms per image  
- **Inference:** 16.0ms per image  
- **Loss Calculation:** 0.0ms per image  
- **Postprocessing:** 3.5ms per image  


## Running Inference and Visualizing YOLO Predictions

##### This code loads a fine-tuned YOLO11n-OBB model and performs batch inference on all images in the "test-images" folder. The model is loaded from a custom-trained checkpoint (best.pt). All images in the folder are processed and resized to 400x400 pixels for uniform display. The results are visualized using Matplotlib in a single row of subplots. The predictions, including detected objects with bounding boxes, will be displayed inside the notebook for easy review.

In [ ]:
# Load the fine-tuned model
model = YOLO("yolo11n-obb.pt")  # load an official model
model = YOLO("runs/obb/train/weights/best.pt")  # load a custom model

# Define folder containing images
image_folder = "test-images"  # Update path
# Get all image paths (supports jpg, png, jpeg)
image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Run inference on all images
results = model(image_paths)  # Batch prediction

# Resize settings
resize_width, resize_height = 400, 400  # Adjust as needed

# Processed images list
processed_images = []
for r in results:
    im_bgr = r.plot()  # Get prediction image (BGR format)
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # Convert BGR to RGB
    im_rgb = im_rgb.resize((resize_width, resize_height))  # Resize
    processed_images.append(im_rgb)

# 📌 Display all images in a row inside Jupyter Notebook
fig, axes = plt.subplots(1, len(processed_images), figsize=(len(processed_images) * 5, 5))

# Ensure axes is iterable for a single image case
if len(processed_images) == 1:
    axes = [axes]

# Show each image in a subplot
for ax, img in zip(axes, processed_images):
    ax.imshow(img)
    ax.axis("off")  # Hide axis

plt.tight_layout()
plt.show()  # 🚀 This ensures images are displayed inside Jupyter Notebook

![YOLO Prediction](output.png)
